# Notes Summarizer App

In [1]:
from openai import OpenAI

def get_response(prompt):
    client = OpenAI()
    response = client.chat.completions.create(model="gpt-3.5-turbo-1106", 
                             messages=
                             [
                                 {"role": "system", "content": "You are a helpful research assistant."},
                                 {"role": "user", "content": prompt}   
                             ],
                             temperature=0.0,
                             n = 1
                             )
    return response.choices[0].message.content

text = """Title: Project Launch Meeting

Date: July 23, 2023

Attendees: John Doe, Jane Smith, Michael Brown, Lisa Johnson

Summary: 

Today we held the kickoff meeting for our new project, CodeName X. The team is very excited to get started. John Doe gave an overview of the project, explaining that the goal is to develop a new AI model that will significantly improve our product's capabilities. The estimated timeline for the project is 18 months, but John stressed that this is a rough estimate and will likely be refined as we delve deeper into the project.

Jane Smith spoke about the resources we have at our disposal, including a robust development team and state-of-the-art AI training facilities. She also mentioned the possibility of bringing in some external consultants for specific tasks, which was met with approval from the rest of the team.

Michael Brown, our lead developer, gave a technical overview of how we might approach the development of the AI model. He proposed a two-stage process: an initial exploratory phase to investigate different methods, followed by a focused development phase where we will construct and fine-tune the final model. 

Lisa Johnson talked about project management and highlighted the importance of maintaining good communication throughout the project. She suggested using Agile methodology to manage the project, which the team agreed upon.

Next Steps:

John will send out a project brief by the end of this week. Jane will look into potential consultants we might want to bring on board. Michael will begin setting up a sandbox for the development team to use during the exploratory phase. Lisa will set up the project in our project management tool and will schedule a project planning meeting for next week.

"""
get_response(f"Summarize this text: {text}")

"The text is a summary of a project launch meeting held on July 23, 2023, for a new project called CodeName X. The attendees included John Doe, Jane Smith, Michael Brown, and Lisa Johnson. John Doe provided an overview of the project's goal to develop a new AI model, with an estimated timeline of 18 months. Jane Smith discussed available resources and the possibility of bringing in external consultants. Michael Brown proposed a two-stage approach for developing the AI model. Lisa Johnson emphasized the importance of communication and suggested using Agile methodology for project management. The next steps include sending out a project brief, looking into potential consultants, setting up a sandbox for the development team, and scheduling a project planning meeting for the following week."

Ok , great we can get a basic summary easy using just ChatGPT, but how about if we want to be able to specify different types of summaries? 
What if the contents we want to summarize are bigger than the context length of our model?

Let's address these issues by using langchain for summarization.

In [2]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import TextLoader
from IPython.display import Markdown, display

With langchain, summarization is as simple as this:

In [3]:
docs = TextLoader("./notes.txt").load()
llm = ChatOpenAI()
chain = load_summarize_chain(llm, chain_type="stuff")
chain.run(docs)

'A project launch meeting was held for the new project, CodeName X. The goal of the project is to develop a new AI model to enhance product capabilities. The estimated timeline is 18 months, subject to refinement. The team discussed available resources, including a development team and AI training facilities, as well as the possibility of hiring external consultants. The lead developer proposed a two-stage process for AI model development. The importance of good communication and Agile methodology for project management was emphasized. Next steps include sending out a project brief, exploring potential consultants, setting up a development sandbox, and scheduling a project planning meeting.'

But as always, we would like to go beyond, why not set up a prompt template to allow for some customization of the summaries?

In [4]:
prompt_template = """Write a concise summary of the following text:

{text}

The summary should be in this style: {style}.
Let's think step by step.

SUMMARY:
"""

prompt_summarization = PromptTemplate.from_template(prompt_template)
chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt_summarization)
summary = chain.run({"input_documents": docs, "style": "bullet points"})
Markdown(summary)

- The kickoff meeting for the new project, CodeName X, was held on July 23, 2023
- The goal of the project is to develop a new AI model to improve product capabilities
- The estimated timeline for the project is 18 months, subject to refinement 
- The team has access to a strong development team and state-of-the-art AI training facilities 
- External consultants may be brought in for specific tasks 
- The proposed approach for development is a two-stage process: an exploratory phase followed by a focused development phase 
- Agile methodology will be used to manage the project 
- Next steps include sending out a project brief, researching potential consultants, setting up a development sandbox, and scheduling a project planning meeting

Ok, let's wrap this into a streamlit app to allow for a nice looking user interface.

[Notes summarize example](https://platform.openai.com/examples/default-meeting-notes-summarizer)

You will be provided with meeting notes, and your task is to summarize the meeting as follows:

-Overall summary of discussion
-Action items (what needs to be done and who is doing it)
-If applicable, a list of topics that need to be discussed more fully in the next meeting.
USER
Meeting Date: March 5th, 2050
Meeting Time: 2:00 PM
Location: Conference Room 3B, Intergalactic Headquarters

Attendees:
- Captain Stardust
- Dr. Quasar
- Lady Nebula
- Sir Supernova
- Ms. Comet

Meeting called to order by Captain Stardust at 2:05 PM

1. Introductions and welcome to our newest team member, Ms. Comet

2. Discussion of our recent mission to Planet Zog
- Captain Stardust: "Overall, a success, but communication with the Zogians was difficult. We need to improve our language skills."
- Dr. Quasar: "Agreed. I'll start working on a Zogian-English dictionary right away."
- Lady Nebula: "The Zogian food was out of this world, literally! We should consider having a Zogian food night on the ship."

3. Addressing the space pirate issue in Sector 7
- Sir Supernova: "We need a better strategy for dealing with these pirates. They've already plundered three cargo ships this month."
- Captain Stardust: "I'll speak with Admiral Starbeam about increasing patrols in that area.
- Dr. Quasar: "I've been working on a new cloaking technology that could help our ships avoid detection by the pirates. I'll need a few more weeks to finalize the prototype."

4. Review of the annual Intergalactic Bake-Off
- Lady Nebula: "I'm happy to report that our team placed second in the competition! Our Martian Mud Pie was a big hit!"
- Ms. Comet: "Let's aim for first place next year. I have a secret recipe for Jupiter Jello that I think could be a winner."

5. Planning for the upcoming charity fundraiser
- Captain Stardust: "We need some creative ideas for our booth at the Intergalactic Charity Bazaar."
- Sir Supernova: "How about a 'Dunk the Alien' game? We can have people throw water balloons at a volunteer dressed as an alien."
- Dr. Quasar: "I can set up a 'Name That Star' trivia game with prizes for the winners."
- Lady Nebula: "Great ideas, everyone. Let's start gathering the supplies and preparing the games."

6. Upcoming team-building retreat
- Ms. Comet: "I would like to propose a team-building retreat to the Moon Resort and Spa. It's a great opportunity to bond and relax after our recent missions."
- Captain Stardust: "Sounds like a fantastic idea. I'll check the budget and see if we can make it happen."

7. Next meeting agenda items
- Update on the Zogian-English dictionary (Dr. Quasar)
- Progress report on the cloaking technology (Dr. Quasar)
- Results of increased patrols in Sector 7 (Captain Stardust)
- Final preparations for the Intergalactic Charity Bazaar (All)

Meeting adjourned at 3:15 PM. Next meeting scheduled for March 19th, 2050 at 2:00 PM in Conference Room 3B, Intergalactic Headquarters.